In [1]:
from pdf2image import convert_from_path

In [41]:
pages = convert_from_path(r"C:\Users\Rajesh Singh\Desktop\data science\1.Python\Project\4_code_basics_projects\2-project_medical_data_extraction\backend\notebooks\docs\patient_details\pd_1.pdf", poppler_path = r"C:\poppler-24.02.0\Library\bin")

In [42]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1867x2000>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2000x1398>]

In [43]:
pages[0].show()

In [44]:
import pytesseract

In [45]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"      # This line of code ensures that pytesseract knows where to find the Tesseract OCR tool{a.k.a.  google tesseract engine} so it can be used for text recognition tasks.
text = pytesseract.image_to_string(pages[0], lang = 'eng')                                          # `image_to_string()` function is used to convert an image into text. can also specify language, if the image has text in diffrent language then can specify that
print(text)

47/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Case of Emergency
m _ a _
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
. : a ee

Chicken Pox (Varicella):

IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine


In [56]:
import numpy as np
import cv2
from PIL import Image

def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)      # `np.array` will convert the image into array form, and our image is a scanned image,it could be in coloured form,so first i have to convert that into greyscale form{black and white form}, which simplifies the image for further processing. For that we use `cv2.COLOR_BGR2GRAY`
    resized = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)  # this involves changing the dimension of the image. Here, image enlarged by 1.5 in both horizontal and vertical directions{this 1.5 value is chosen after trial and error}
    processed_image = cv2.adaptiveThreshold(
        resized,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 
        61,
        11
    )
    return processed_image

In [47]:
img = preprocess_image(pages[0])
Image.fromarray(img).show()

In [48]:
# now will apply the teseract to extract text from the post processed image
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
text = pytesseract.image_to_string(img, lang = 'eng')
print(text)

17/12/2020

Patient Medical Record . : :

Patient Information

Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight:
9264 Ash Dr 95
New York City, 10005 a
United States Height:
190
In Case of Emergency
en oe
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
I a
Chicken Pox (Varicella): Measies:
IMMUNE IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine


**Extract name**

In [49]:
import re

pattern = 'Patient Information(.*?)\(\d{3}\)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['\n\nBirth Date\nKathy Crawford May 6 1972\n']

In [50]:
matches[0].strip()

'Birth Date\nKathy Crawford May 6 1972'

In [51]:
match = matches[0].replace("Birth Date","").strip()
match

'Kathy Crawford May 6 1972'

In [52]:
pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'

date_matches = re.findall(pattern, match)
date = date_matches[0][0]
date

'May 6 1972'

In [53]:
match.replace(date, '').strip()

'Kathy Crawford'

In [57]:
def remove_noise_from_name(name):
    name = name.replace('Birth Date','').strip()
    date_pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches = re.findall(date_pattern, name)
    
    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date, '').strip()
    
    return name


In [58]:
name = '\n\nBirth Date\nKathy Crawford May 6 1972\n'

name = remove_noise_from_name(name)
name

'Kathy Crawford'

**Extract Phone**

In [63]:
pattern = 'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'  ## note here i used `?` becase i do not want to be greedy while matching because if would not have used it thn it wouldhave matched everything not only until the parenthesis closed three numbers.

matches = re.findall(pattern, text, flags=re.DOTALL)
matchess

[('\n\nBirth Date\nKathy Crawford May 6 1972\n', '(737) 988-0851')]

In [64]:
matches[0][1]

'(737) 988-0851'

**Extract Vaccine**

In [65]:
pattern = 'Have you had the Hepatitis B vaccination\?.*(Yes|No)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['No']

**Extract Medical Problems**

In [66]:
pattern = 'List any Medical Problems \(asthma, seizures, headaches\):(.*)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'Migraine'